### TODO

- effectuer les similarités entre technolectes de questions en questions et de réponses en réponses.
- faire de même entre une question et ses réponses ?


### Ce qui a été fait

J'ai essayé de faire une première classification par règles. Pour chaque question, je crée des similarités cosinus au mot entre cette question et les autres questions et entre sa réponse et les autres réponses. Toujours pour chaque question, je ne garde que les résultats supérieurs à 0.8 (valeur à modifier par la suite ?). 

Suite à cela, j'ai favorisé une approche plus normée. Les questions sont "bruitées" quand on parle de similarité : beaucoup de termes en commun ne donnant aucun indice sur la réponse à trouver. De plus, une similarité au mot semble peu utile. Il faudrait conserver **exclusivement les technolectes appartenant au champs lexical de la médecine aussi bien dans les questions que dans les réponses**. Avec ces technolectes, on peut ensuite faire de similarités au caractère, plus pertinentes selon moi dans ce contexte.

Les règles de nettoyage sont les suivantes :

- suppression des mots vides, de la ponctuation et des mots d'une longueur de 1. On ignore la case SANS LA SUPPRIMER pour autant.
- on parcours une liste de mots du français (deux listes trouvées, une de 20 000 et plus occurrences, l'autre de 300 000 et plus occurrences) et pour chaque mot reconnu on l'exclu du vocabulaire des technolectes.
- à la fin, on a une liste de mots qui ne sont pas retrouvés dans la liste de mots du français utilisée, la plupart étant des technolectes propres au champs lexical de la médecine.


### Remarques personnelles sur la tâche

Je distingue deux approches au problème :

- une approche visant à savoir si le jeu de données peut se suffire afin d'obtenir de meilleurs résultats que ceux obtenus dans le papier "source" : X questions nous donnent Y indices sur un concept médical, donc on a des liens entre mots clefs au sein du corpus qu'on exploite par la suite.
- une approche visant à créer ou exploiter une base de données médicale suffisement exhaustive afin de permettre une reconnaissance de termes clef entre eux : terme clef A de la question est plus souvent lié au terme B de la réponse X dans un contexte médical.


In [1]:
#Imports et fonctions

import json
import pandas as pd
import string as strii

#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy
sp = spacy.load("fr_core_news_sm")

def removePunct(string):
    return string.translate(str.maketrans(strii.punctuation, ' '*len(strii.punctuation))) #map punctuation to space

def tokenizer(string):
    spacy_object = sp(string)
    return [word.text for word in spacy_object if word.is_stop == False] #and word.pos_ != "PUNCT" and word.pos_ != "NUM"

def makeVoc(liste):
    voc = []
    for item in liste:
        for word in tokenizer(removePunct(item)):
            if word not in voc:
                voc.append(word)
    return [w for w in voc if len(w) > 1]

def vectorizer(q,liste_q,ngram_range=(1,1),analyzer="word"): #analyzer{‘word’, ‘char’, ‘char_wb’}
    #vect = TfidfVectorizer()
    vect = CountVectorizer(lowercase=True,ngram_range=ngram_range, analyzer=analyzer)
    liste_q_vect = vect.fit_transform(liste_q) #vectoriser cet ensemble à part ? gain de temps
    liste_q_term_matrix = liste_q_vect.toarray()
    q_vect = vect.transform([q]).toarray()
    return q_vect,liste_q_term_matrix
    
def cosinus(q_vect,liste_q_term_matrix):
    return cosine_similarity(liste_q_term_matrix,q_vect)

def writeJson(path,data):
    with open(path,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4,ensure_ascii=False)
        
def openJson(path):
    with open(path,'r',encoding='utf-8') as f:
        data = json.load(f)
    return data       

In [2]:
#Chargement du jeu de données et ajout d'une concaténation des réponses

df = pd.read_csv("../data/csv/train.csv",delimiter=";")

In [3]:
#Création du vocabulaire cible composé de technolectes

liste = "input/liste.de.mots.francais.frgut.txt"
with open(liste,'r',encoding='utf-8') as f:
     liste_mots = [line.rstrip('\n').lower() for line in f]
liste_keywords = openJson("output/corpusRef/keywordsMesh.json")

voc_global = set(makeVoc(df["question"]))

voc_medical = []
for word in voc_global:
    if len(word) > 1:
        if word.lower() not in liste_mots:
            voc_medical.append(word)
        elif word.lower() in liste_keywords:
            voc_medical.append(word)

writeJson('output/classification/vocabulaire médical.json',voc_medical)

In [6]:
#Première tâche de similarité : test en ngram char_wb

voc_medical = openJson("output/classification/vocabulaire médical.json")
dic = {}

ids = df["id"]
questions = df["question"]

ids_merged_q = []
merged_q = []

for i in range(len(df)):
    voc_m_q = [w for w in removePunct(questions[i]).split() if w in voc_medical]
    if len(voc_m_q) > 0:
        merged_q.append(" ".join(voc_m_q))
        ids_merged_q.append(ids[i])

for i,mq in enumerate(merged_q):
    dic[ids_merged_q[i]] = {}
    vect_mq,vect_merged_q = vectorizer(mq,merged_q,ngram_range=(1,3),analyzer="char_wb")
    cos = [list(s) for s in cosinus(vect_mq,vect_merged_q)]
    for j,res in enumerate(cos):
        dic[ids_merged_q[i]][ids_merged_q[j]] = cos[j][0]
    
writeJson("sims.json",dic)

In [7]:
#Observation des résultats : classification des paires q/r similaires selon un seuil

data = openJson("sims.json")
similarities_classes = []

for ib, subdic in data.items():
    sim_classe = []
    for k,v in subdic.items():
        if v > 0.8:
            sim_classe.append(k)
    similarities_classes.append(sim_classe)
    
#similarities_classes_clean = []
#for l in similarities_classes:
#    if len(l) > 1:
#        similarities_classes_clean.append(l)
    
#on sauvegarde les clusters en supprimant les classes identiques ET LES CLASSES D'UN ELEMENT
writeJson("output/classification/id_classes.json",[list(item) for item in set(tuple(row) for row in similarities_classes)]) 

In [8]:
#Si on souhaite observer le rendu de la classification

data = openJson("output/classification/id_classes.json")

questions_list = []
for l in data:
    new_l = []
    for i in range(len(df)):
        if df["id"][i] in l:
            new_l.append(df["question"][i])
    questions_list.append(new_l)
    
writeJson("output/classification/content_classes.json",questions_list) 

In [11]:
"""
id_classes = openJson("output/classification/id_classes.json")
df = pd.read_csv("../corpus_builder/new_csv.csv",delimiter=",",index_col=0)

related_lines = []

for i in range(len(df)):
    rl = []
    for liste in id_classes:
        if df["id"][i] in liste:
            rl = rl + liste
    related_lines.append(list(set(rl)))
    
df["questions_related_to"] = related_lines
df.to_csv("../corpus_builder/new_csv.csv",index=True)
"""

print(len(df))
print(len(data))

2171
1796
